In [15]:
import os
import sys

FS_MOL_CHECKOUT_PATH = os.path.abspath('../')

os.chdir(FS_MOL_CHECKOUT_PATH)
sys.path.insert(0, FS_MOL_CHECKOUT_PATH)
from fs_mol.data import FSMolDataset, DataFold

dataset = FSMolDataset.from_directory('/FS-MOL/datasets/fs-mol/', num_workers=0)

tasks = dataset.get_task_reading_iterable(data_fold=DataFold.TRAIN)

task = next(iter(tasks))

fsmol_molecules = task.samples[:2]

fsmol_molecules


[MoleculeDatapoint(task_name='CHEMBL2213433', smiles='Cc1c(OCC(=O)c2ccccc2)ccc2c1oc(=O)c1ccccc12', graph=GraphData(node_features=array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,

In [16]:

from fs_mol.data.fsmol_batcher import FSMolBatcher
from fs_mol.data.protonet import batcher_add_sample_fn, batcher_finalizer_fn, batcher_init_fn


batcher = FSMolBatcher(
    10,
    1000,
    1000,
    init_callback=batcher_init_fn,
    per_datapoint_callback=batcher_add_sample_fn,
    finalizer_callback=batcher_finalizer_fn
)

batches = list(batcher.batch(fsmol_molecules))

fsmol_batch = batches[0][0]

In [17]:
from fs_mol.custom.utils import convert_to_pyg_graph
from torch_geometric.loader.dataloader import Collater

pyg_mols = list(map(convert_to_pyg_graph, task.samples[:2]))

pyg_batch = Collater(follow_batch=None, exclude_keys=None)(pyg_mols)

In [18]:
import torch
from fs_mol.models.protonet import GraphFeatureExtractor, GraphFeatureExtractorConfig
from fs_mol.modules.pyg_gnn import PyG_GraphFeatureExtractor
from fs_mol.utils.torch_utils import torchify

f_model = GraphFeatureExtractor(GraphFeatureExtractorConfig()) # 10137120

m_model = PyG_GraphFeatureExtractor(GraphFeatureExtractorConfig()) # 10137120


for n, p in f_model.named_parameters():
    with torch.no_grad():
        p.fill_(0.01)

for n, p in m_model.named_parameters():
    with torch.no_grad():
        p.fill_(0.01)


f_res = f_model(torchify(batches[0][0], device=torch.device('cpu')))

In [19]:
m_res = m_model(pyg_batch)


torch.allclose(f_res, m_res)

True